# Code for applying template matching method to find sleep drive correlates

Raw data is available in GEO under accession code GSE221239
Processed loom file is available on scope https://scope.aertslab.org/#/Fly_Brain_Sleep/Fly_Brain_Sleep%2FFly_Sleep.loom/

In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
import pickle
import os
import scipy.io
import scanpy as sc
import anndata as ad
import seaborn as sns
from statsmodels.stats import multitest

%config InlineBackend.figure_format = 'retina'

In [2]:
sc.settings.verbosity = 3             # verbosity: errors (0), warnings (1), info (2), hints (3)
sc.logging.print_versions()
sc.settings.set_figure_params(dpi=120)

-----
anndata     0.8.0
scanpy      1.9.1
-----
PIL                 8.1.0
backcall            0.1.0
cached_property     1.5.2
cffi                1.15.0
cloudpickle         2.0.0
constants           NA
cycler              0.10.0
cython_runtime      NA
cytoolz             0.11.2
dask                2022.02.0
dateutil            2.8.1
decorator           4.4.1
fsspec              2022.3.0
h5py                3.6.0
highs_wrapper       NA
ipykernel           5.1.3
ipython_genutils    0.2.0
jedi                0.15.2
jinja2              2.10.3
joblib              1.0.0
kiwisolver          1.3.1
llvmlite            0.35.0
markupsafe          1.1.1
matplotlib          3.5.3
more_itertools      NA
mpl_toolkits        NA
natsort             8.0.2
numba               0.52.0
numexpr             2.8.1
numpy               1.19.5
packaging           21.3
pandas              1.3.4
parso               0.5.2
pexpect             4.7.0
pickleshare         0.7.5
pkg_resources       NA
prompt_toolkit      

# 1. load adata.h5ad

In [3]:
os.chdir('/lustre1/project/stg_00003/groups/SHLI/')

In [4]:
adata = sc.read_h5ad('adata_08082022.h5ad')

In [5]:
adata.obs

,Age,Condition,Genotype,Run,Singlet,Sleep_Stage,Treatment,doublet_scores,n_counts,n_genes,...,Neurotransmitter,all_res8,HRG_sqrt,CRG_sqrt,Cluster_ID_res8_240622,Cluster_ID_res8_070722,Cluster_ID_res8_080822,ring_3clusters,neuron_glia,neuron_glia_pos
index,,,,,,,,,,,,,,,,,,,,,
AAACCCAAGGAGGCAG-fc9729__DGRP_Mix_Sleep_Deprivation_1-20180419,6-9 days,ZT 8 wake,line_88,20180419,True,ZT_8,Wake,0.042720,7702.0,2211,...,Unknown,nonPAM,10.440307,1.914854,nonPAM,nonPAM,nonPAM,NaN,neuron,neuron
AAACCCAAGGGTCACA-fc9729__DGRP_Mix_Sleep_Deprivation_1-20180419,6-9 days,ZT 20 sleep,line_441,20180419,True,ZT_20,Sleep,0.027581,8043.0,2246,...,Cholinergic,201,5.385165,0.000000,201,201,201,NaN,neuron,neuron
AAACCCACAAATACAG-fc9729__DGRP_Mix_Sleep_Deprivation_1-20180419,6-9 days,ZT 8 wake stimulation,line_313,20180419,True,ZT_8,Wake,0.053623,655.0,411,...,Unknown,ALG-,3.316625,0.000000,133,133,133,NaN,glia,133
AAACCCACAAGCACCC-fc9729__DGRP_Mix_Sleep_Deprivation_1-20180419,6-9 days,ZT 20 sleep,line_441,20180419,True,ZT_20,Sleep,0.027581,3774.0,1492,...,Cholinergic,90,6.782330,4.281744,90,90,90,NaN,neuron,neuron
AAACCCACAAGTGGAC-fc9729__DGRP_Mix_Sleep_Deprivation_1-20180419,6-9 days,ZT 20 sleep,line_441,20180419,True,ZT_20,Sleep,0.058220,15036.0,3112,...,Cholinergic,34,3.162278,4.358899,34,34,34,NaN,neuron,neuron
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TTTGGTTTCTGTAAGC-fcae49__S3_10x_240920-20200924,7 days,ZT 8 gab,line_287,20200924,True,ZT_8,Sleep,0.036336,1589.0,797,...,Cholinergic,138,0.000000,0.000000,138,138,138,NaN,neuron,neuron
TTTGTTGAGGAACTCG-fcae49__S3_10x_240920-20200924,6-8 days,ZT 8 gab,line_441,20200924,True,ZT_8,Sleep,0.010993,1490.0,809,...,Glutamatergic,67,0.000000,5.972158,67,67,67,NaN,neuron,neuron
TTTGTTGCAACCAATC-fcae49__S3_10x_240920-20200924,7 days,ZT 8 wake,line_379,20200924,True,ZT_8,Wake,0.012016,4453.0,1483,...,Unknown,26,6.928203,6.952218,26,26,26,NaN,neuron,neuron


# 2. define template

In [6]:
# sleep drive conditions used in the template in order from low to high sleep drive
template_keys = [
    'ZT 8 gab',
    'ZT 20 sleep',
    'ZT 14 sleep',
    'ZT 14 SD',
    'ZT 20 sleep deprivation',
    'ZT 2 sleep drive 14h SD',
    'ZT 8 20h SD'
]

In [7]:
for cond in template_keys:
    if cond not in set(adata.obs['Condition'].values):
        print(cond)

In [8]:
template_vals = np.linspace(0, 1, len(template_keys))

In [9]:
# assign values between 0 and 1 to ordered sleep drive conditions
template_dict = {k: v for k, v in zip(template_keys, template_vals)}

In [10]:
template_dict

{'ZT 8 gab': 0.0,
 'ZT 20 sleep': 0.16666666666666666,
 'ZT 14 sleep': 0.3333333333333333,
 'ZT 14 SD': 0.5,
 'ZT 20 sleep deprivation': 0.6666666666666666,
 'ZT 2 sleep drive 14h SD': 0.8333333333333333,
 'ZT 8 20h SD': 1.0}

In [9]:
# define dict with cluster numbers
cluster_dict = dict.fromkeys(adata.obs['Cluster_ID_res8_080822'], "clusters")

In [14]:
cluster_dict.keys()

dict_keys(['nonPAM', '201', '133', '90', '34', '13', '80', 'y', '4', '57', 'EG_2', '9', '32', '36', '138', '28', '193', '117', '143', '46', '30', '126', '130', '26', 'ab', '97', '196', '51', '10', '123', '56', '43', '29', '77', 'adPN', '55', '38', '35', '45', '98', '115', '61', '22', '74', '49', '186', '127', '105', '58', '72', '70', '20', '69', '128', '131', '116', '81', '87', '44', '78', 'ALG', '11', '52', '41', '68', '99', 'EG_1', '8', '110', '157', '7', '23', 'abp', '104', '112', 'PG', '124', 'dFB', '158', '5-HT', '27', '16', '119', '47', 'PAM', '18', '94', '48', '182', '0', '142', '14', '86', 'CXG', '191', '148', '63', '31', '33', '163', '176', '199', '40', '177', '144', '24', '125', '113', '162', '187', '39', '96', '178', '172', '106', '62', 'ring_B', '91', '108', '50', '102', '73', 'ring_A', '54', '122', '53', '161', '66', 'PB45', '114', '67', '88', '154', '205', '59', '93', 'R5', '203', '179', '147', '107', '65', '141', '174', '42', '76', '89', '149', '83', '175', '139', '60', 

3. create meta matrix per cluster

In [15]:
ct_mms = {}
for cell_type in cluster_dict.keys():
    meta_matrix = pd.DataFrame(columns = template_keys, index = adata.raw.var_names)
    try:
        for cond in meta_matrix.columns:
            subset = adata[np.logical_and(adata.obs['Condition'] == cond, adata.obs['Cluster_ID_res8_080822'] == cell_type)]
            meta_matrix[cond] = np.array(subset.raw.X.mean(axis=0))[0]
        ct_mms[cell_type] = meta_matrix
    except ZeroDivisionError: 
        print(f'Cell type fails {cell_type}')
        continue

Cell type fails 13
Cell type fails 193
Cell type fails 117
Cell type fails 125
Cell type fails 205
Cell type fails 202
Cell type fails 3
Cell type fails 195
Cell type fails 5
Cell type fails 188
Cell type fails 208
Cell type fails 6
Cell type fails 25
Cell type fails 209
Cell type fails 75
Cell type fails 103
Cell type fails 21
Cell type fails 15
Cell type fails 2


In [12]:
#clusters with no cells in one of the conditions
no_cells_list = ['13', '193', '117', '125', '205', '202', '3', '195', '5', '188', '208', '6', '25', '209', '75', '103', '21', '15', '2']

# 4. calculate pearson r and adj pvalue

In [ ]:
ct_corrs = {}
for cell_type in cluster_dict.keys():
    if cell_type in no_cells_list: continue
    meta_matrix = ct_mms[cell_type]
    corrs = pd.DataFrame(index = meta_matrix.index, columns = ['Pearson_r2', 'p-value'])
    for g, vals in meta_matrix.iterrows():
        rp = scipy.stats.pearsonr(vals, template_vals)
        corrs.loc[g] = rp
    pa = multitest.multipletests(corrs['p-value'], method='fdr_bh', alpha=0.05)
    corrs['p-adj'] = pa[1]
    print(f'{cell_type}, {sum(pa[0])}')
    ct_corrs[cell_type] = corrs

/data/leuven/334/vsc33403/jupyter/envs/scanpy/lib/python3.7/site-packages/scipy/stats/stats.py:3508: PearsonRConstantInputWarning: An input array is constant; the correlation coefficent is not defined.
  warnings.warn(PearsonRConstantInputWarning())


nonPAM, 5181
201, 6229
133, 5044
90, 3300
34, 819
80, 5081
y, 0
4, 0
57, 0
EG_2, 2725
9, 188
32, 4195
36, 0
138, 0
28, 0
143, 71
46, 0
30, 0
126, 5234
130, 5079
26, 5046
ab, 0
97, 0
196, 5066
51, 0
10, 0
123, 23
56, 5017
43, 48
29, 257
77, 823
adPN, 0
55, 0
38, 0
35, 0
45, 0
98, 0
115, 178
61, 430
22, 0
74, 0
49, 0
186, 6970
127, 5121
105, 521
58, 445
72, 5086
70, 0
20, 0
69, 0
128, 442
131, 5155
116, 1066
81, 1763
87, 0
44, 5064
78, 5493
ALG, 4265
11, 0
52, 0
41, 355
68, 0
99, 0
EG_1, 2574
8, 0
110, 6003
157, 5053
7, 0
23, 0
abp, 759
104, 0
112, 0
PG, 0
124, 0
dFB, 6800
158, 0
5-HT, 0
27, 78
16, 1137
119, 1218
47, 5056
PAM, 382


# 5. filter genes with adjusted p-value < 0.05

In [67]:
padj = {}

In [68]:
for cell_type in cluster_dict.keys():
    if cell_type in no_cells_list: continue
    corrs = ct_corrs[cell_type]
    low = corrs[corrs['p-adj'] <= 0.05]
    low_sorted = low.sort_values('Pearson_r2', ascending=True)
    padj[cell_type] = low_sorted

# 6. filter out batch effect genes

In [28]:
batch_effect_genes = pd.read_excel('/lustre1/project/stg_00003/groups/SHLI/batch_effects.xlsx', sheet_name = "all")

In [29]:
batch_effect_genes_list = list(batch_effect_genes["gene"])
batch_effect_genes_list

['GstE4',
 'cato',
 'CG3117',
 'CR45600',
 'CG31157',
 'CG6912',
 'fw',
 'sdic3',
 'CR45175',
 'CR44841',
 'CG30428',
 'RhoGEF2',
 'Rs1',
 'CG9975',
 'CG7091',
 'Hsc70-2',
 'CG1678',
 'CG34220',
 'CG34324',
 'CG14125',
 'Muc68D',
 'Eh',
 'CR41609',
 'CG14645',
 'CG3906',
 'Peritrophin-15a',
 'CG11672',
 'Cyp4g1',
 'CG4783',
 'CG5399',
 'alrm',
 'nSyb',
 'CadN',
 'brp',
 'noe',
 'Hug',
 'Trissin',
 'Dsk',
 'CR40469',
 'mt:srRNA',
 'Ms',
 'Ilp2',
 'CG12239']

In [71]:
df = pd.DataFrame.from_dict(low_padj, orient = "index").sort_index().stack().reset_index(level=1,drop=True).reset_index()
df

,index,0
0,0,CG11208
1,0,RhoGEF3
2,1,GlcAT-I
3,1,CG30428
4,1,Rx
...,...,...
5369,nonPAM,Lsm11
5370,nonPAM,galectin
5371,nonPAM,CG17359
5372,ring_A,Ranbp16


In [72]:
df.columns = ['cluster', 'gene']

In [132]:
df_nobatch = df[~df['gene'].isin(batch_effect_genes_list)]
df_nobatch_len=pd.DataFrame.from_dict(df_nobatch['cluster'].value_counts().to_dict(), orient="index").reset_index(level=0)
df_nobatch_len.columns = ['cluster', 'n_HAG']
df_nobatch_len

,cluster,n_HAG
0,85,148
1,109,139
2,190,123
3,dFB,121
4,Oct,119
...,...,...
122,42,1
123,184,1
124,129,1
125,128,1


In [76]:
all_clusters_list = list(set(adata.obs['Cluster_ID_res8_080822'].values.tolist()))
len(all_clusters_list)

214

In [87]:
HAG_nobatch = list(df_nobatch_len['cluster'])
len(HAG_nobatch)

127

In [79]:
combined_HAG = HAG_nobatch + no_cells_list
len(combined_HAG)

146

In [82]:
no_HAG = list(set(all_clusters_list) - set(combined_HAG))
len(no_HAG)

68

In [121]:
no_HAG_df = pd.DataFrame(no_HAG)

In [122]:
no_HAG_df['cluster']=no_HAG_df.index
no_HAG_df.columns = ['cluster', 'n_HAG']
no_HAG_df['n_HAG'] = 0
no_HAG_df

,cluster,n_HAG
0,69,0
1,57,0
2,46,0
3,23,0
4,111,0
...,...,...
63,24,0
64,99,0
65,5-HT,0
66,39,0


In [133]:
HAG = pd.concat([df_nobatch_len, no_HAG_df])
HAG

,cluster,n_HAG
0,85,148
1,109,139
2,190,123
3,dFB,121
4,Oct,119
...,...,...
0,24,0
0,99,0
0,5-HT,0
0,39,0


# 7. save gene list by cluster with all columns (p_adj etc.)

In [5]:
for key, df in padj.items():
    df.reset_index(inplace=True, drop=False)
    df.rename(columns={'index': 'gene'}, inplace=True)# Drop=False keeps the index as a normal column
    padj[key] = df

In [6]:
# convert dict to a table with columns: cluster, gene, r2, p-value, p-adj
table_data = []

for key, df in padj.items():
    # Get the DataFrame values as a list of lists
    df_values = df.values.tolist()
    # Create a row with the key followed by the DataFrame's values
    for row_values in df_values:
        row_data = [key] + row_values
        table_data.append(row_data)

# Get the column names from the first DataFrame in the dictionary
column_names = ['cluster'] + list(padj[next(iter(padj))].columns)

In [7]:
df = pd.DataFrame(table_data, columns=column_names)
df

,cluster,gene,Pearson_r2,p-value,p-adj
0,nonPAM,mRpS25,-0.976229,0.000165,0.019895
1,nonPAM,CG11885,-0.963960,0.000464,0.000928
2,nonPAM,jet,-0.960490,0.000583,0.001165
3,nonPAM,TBCB,-0.957760,0.000688,0.001374
4,nonPAM,YL-1,-0.953599,0.000869,0.001737
...,...,...,...,...,...
5369,207,IntS10,0.902368,0.005424,0.010358
5370,207,CG8078,0.902883,0.005354,0.009804
5371,207,Scox,0.902903,0.005352,0.010301
5372,207,Hop,0.903686,0.005247,0.009608


In [74]:
filtered_df = df[~df['gene'].isin(batch_effect_genes_list)]
filtered_df

,cluster,gene,Pearson_r2,p-value,p-adj
0,nonPAM,mRpS25,-0.976229,0.000165,0.019895
1,nonPAM,CG11885,-0.963960,0.000464,0.000928
2,nonPAM,jet,-0.960490,0.000583,0.001165
3,nonPAM,TBCB,-0.957760,0.000688,0.001374
4,nonPAM,YL-1,-0.953599,0.000869,0.001737
...,...,...,...,...,...
5369,207,IntS10,0.902368,0.005424,0.010358
5370,207,CG8078,0.902883,0.005354,0.009804
5371,207,Scox,0.902903,0.005352,0.010301
5372,207,Hop,0.903686,0.005247,0.009608


In [76]:
filtered_df.to_csv('/staging/leuven/stg_00003/groups/SHLI/PTM/sign_sleepdrive.csv', index=False)